In [ ]:
# Data analysis packages:
import pandas as pd
import numpy as np
# pd.set_option('display.float_format', lambda x: '%.5f'%x)
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Metadata

In [ ]:
metadata = pd.read_csv("data/building_metadata.csv")
print(metadata.shape)
print(metadata['site_id'].unique())
print(metadata.info(memory_usage='deep'))
metadata.head(3)

### Weather data

In [ ]:
weather_train = pd.read_csv("data/weather_train.csv", parse_dates=['timestamp'])
print(weather_train.shape)
print(weather_train['site_id'].unique())
print(weather_train.info(memory_usage='deep'))
weather_train.head(3)

In [ ]:
train = pd.read_csv("data/train.csv", parse_dates=['timestamp'])
print(train.shape)
# print(train['site_id'].unique())
# train.meter_reading.value_counts()
train.info()

### Dealing with missing values 

In [ ]:
missing_weather = pd.DataFrame(weather_train.isna().sum()/len(weather_train),columns=["Weather_Train_Missing_Pct"])
# missing_weather["Weather_Test_Missing_Pct"] = weather_test.isna().sum()/len(weather_test)
missing_weather

In [ ]:
# missing values 
metadata.isna().sum()/len(metadata)

In [ ]:
train.isna().sum()/len(train)

In [ ]:
train.describe().T

In [ ]:
print(train['meter'].value_counts())
train.head(3)

In [ ]:
train['meter'].replace({0:"Electricity",1:"ChilledWater",
                       2:"Steam",3:"HotWater"}, inplace=True)
train.head(3)

In [ ]:
import seaborn as sns 
# sns.set(style='darkgrid')
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
# sns.set_context('talk')
sns.countplot(train['meter'], order=train['meter'].value_counts().index)
plt.title('Distribution of Meter Id Code')
plt.xlabel('Meter Id Code')
plt.ylabel('Frequency');

In [ ]:
print ("There are {} unique Buildings in the training data".format(train['building_id'].nunique()))

In [ ]:
train.building_id.value_counts(dropna=False).head(3)

Like it is mentioned in the competition description, each building may or may not have all 4 meter

In [ ]:
train[train['building_id'] == 1094]['meter'].unique()

In [ ]:
train.head(1)

In [ ]:
train.groupby('meter')['meter_reading'].agg(['min','max','mean','median', 'count', 'std'])

In [ ]:
for df in [train]:
    df['month'] = df['timestamp'].dt.month.astype("uint8")
    df['day_of_month'] = df['timestamp'].dt.day.astype("uint8")
    df['day_of_week'] = df['timestamp'].dt.dayofweek.astype("uint8")
    df['hour'] = df['timestamp'].dt.hour.astype("uint8")

In [ ]:
train.drop(columns=['Month', 'DayOfMonth', 'DayOfWeek', 'Hour'], inplace=True)
train.head(1)

In [ ]:
train.groupby(['meter', 'month'])['meter_reading'].agg(['max', 'mean', 'median', 'count', 'std'])

In [ ]:
train.groupby(['meter', 'day_of_week'])['meter_reading'].agg(['max', 'mean', 'median', 'count', 'std'])

In [ ]:
train.head(1)

In [ ]:
train['meter_reading'].describe()

In [ ]:
sns.distplot(np.log1p(train['meter_reading']), kde=False)
plt.title('Distribution of log of meter reading variable')

In [ ]:
sns.boxplot(train[train['meter'] == "Electricity"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: Electricity");
# We can see a few outliers here. 

In [ ]:
sns.boxplot(train[train['meter'] == "ChilledWater"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: ChilledWater");
# Not many outliers here.

In [ ]:
sns.boxplot(train[train['meter'] == "HotWater"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: HotWater");
# We can see a single value that is way off from the rest. 

In [ ]:
sns.boxplot(train[train['meter'] == "Steam"]['meter_reading']);
plt.title("Boxplot of Meter Reading Variable for the Meter Type: Steam");
# We can see a few outliers here. 

### Converting the dependent variable to logarithmic scale

In [ ]:
train['meter_reading'] = np.log1p(train['meter_reading'])


In [ ]:
sns.distplot(train[train['meter'] == "Electricity"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Electricity")

In [ ]:
sns.distplot(train[train['meter'] == "ChilledWater"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Chilledwater")

In [ ]:
sns.distplot(train[train['meter'] == "Steam"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Steam")

In [ ]:
sns.distplot(train[train['meter'] == "HotWater"]['meter_reading'],kde=False)
plt.title("Distribution of Meter Reading per MeterID code: Hotwater")

In [ ]:
train.head(1)

## Metadata 

In [ ]:
metadata.info()
# Missing values in year_built and floor_count variables

In [ ]:
metadata.head(3)

In [ ]:
cols = ['site_id','primary_use','year_built','floor_count','building_id']
for col in cols:
    print ("Number of Unique Values in the {} column are:".format(col),metadata[col].nunique())

In [ ]:
cols = ['site_id','primary_use','floor_count']
for col in cols:
    print ("Unique Values in the {} column are:".format(col),metadata[col].unique())
    print ("\n")

In [ ]:
sns.countplot(metadata['site_id'])
plt.title("Count of Site_id in the Metadata table")
plt.xlabel("Site_Id")
plt.ylabel("Count")

In [ ]:
plt.figure(figsize=(8,6))
metadata['primary_use'].value_counts().sort_values().plot(kind='bar')
plt.title("Count of Primary_Use Variable in the Metadata table")
plt.xlabel("Primary Use")
plt.ylabel("Count")
plt.xticks(rotation=90)
# Education, Office, Entertainment/Public Assembly, Public Services, Lodging/Residential form the bulk of Primary Use

In [ ]:
metadata['square_feet'].describe()

In [ ]:
sns.boxplot(metadata['square_feet'])

In [ ]:
metadata['square_feet'] = np.log1p(metadata['square_feet'])

In [ ]:
sns.distplot(metadata['square_feet'])
plt.title("Distribution of Square Feet variable of Metadata Table")
plt.xlabel("Area in Square Feet")
plt.ylabel("Frequency")
# Looks like a normal distribution distribution

In [ ]:
sns.boxplot(metadata['square_feet'])
plt.title("Box Plot of Square Feet Variable")
# There are a few outliers visible

In [ ]:
metadata.groupby('primary_use')['square_feet'].agg(['mean','median','count']).sort_values(by='count')
# Parking has the highest average are although the count is less.
# Education has the highest count as can be seen in the countplot above.

In [ ]:
metadata['year_built'].value_counts().sort_values().plot(kind='bar',figsize=(15,6))
plt.xlabel("Year Built")
plt.ylabel("Count")
plt.title("Distribution of Year Built Variable")

In [ ]:
metadata['floor_count'].value_counts(dropna=False).sort_index().plot(kind='bar',figsize=(8,6))
plt.xlabel("Number of Floors")
plt.ylabel("Count of Buildings")
# Lot of missing values here as well
# Maximum number of floors is 26

In [ ]:
metadata.groupby('floor_count')['square_feet'].agg(['count','mean','median']).sort_values(by='count')

In [ ]:
metadata.groupby('primary_use')['square_feet'].agg(['count','mean','median']).sort_values(by='count')

In [ ]:
metadata['year_built'].fillna(-999, inplace=True)
metadata['year_built'] = metadata['year_built'].astype('int16')
metadata['floor_count'].fillna(-999, inplace=True)
metadata['floor_count'] = metadata['floor_count'].astype('float32')

In [ ]:
metadata.head(3)

# Weather Data (Train)

In [ ]:
weather_train.head(3)

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']
for col in cols:
    print (" Minimum Value of {} column is {}".format(col,weather_train[col].min()))
    print (" Maximum Value of {} column is {}".format(col,weather_train[col].max()))
    print ("----------------------------------------------------------------------")

In [ ]:
weather_train.isna().sum()/len(weather_train)

In [ ]:
weather_train[['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']].describe()

In [ ]:
print(weather_train['timestamp'].min())
print(weather_train['timestamp'].max())
print(len( weather_train['timestamp']))
# This data is from 1st Jan to 31st Dec 2016, similar to the timestamp of the training data

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']
for ind,col in enumerate(weather_train[cols]):
    plt.figure(ind)
    sns.distplot(weather_train[col].dropna())

In [ ]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_speed']
for ind,col in enumerate(weather_train[cols]):
    plt.figure(ind)
    sns.boxplot(weather_train[col].dropna())

# Merging Data

In [ ]:
train.head(3)

In [ ]:
metadata.head(3)

In [ ]:
weather_train.dtypes

In [ ]:
# %%time
train_data = pd.merge(train,metadata,on='building_id',how='left')
# test  = pd.merge(test,metadata,on='building_id',how='left')
print ("Training Data Shape {}".format(train_data.shape))
# print ("Testing Data Shape {}".format(test.shape))
# gc.collect()

In [ ]:
train_data.dtypes

In [ ]:
# %%time
train = pd.merge(train_data,weather_train,on=['site_id','timestamp'],how='left')
# test  = pd.merge(test,weather_test,on=['site_id','timestamp'],how='left')
print ("Training Data Shape {}".format(train.shape))
# print ("Testing Data Shape {}".format(test.shape))
# gc.collect()

In [ ]:
train.head(3)

In [ ]:

cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']
for col in cols:
    train[col].fillna(train[col].mean(),inplace=True)
#     test[col].fillna(test[col].mean(),inplace=True)
    
del metadata, weather_train#, weather_test

In [ ]:
train.columns

In [ ]:
# %%time
for df in [train]:#, test]:
    df['weekend'] = np.where((df['day_of_week'] == 5) | (df['day_of_week'] == 6), 1, 0)
    df['air_temperature'] = df['air_temperature'].astype('float16')
    df['cloud_coverage'] = df['cloud_coverage'].astype("float16")
    df['dew_temperature'] = df['dew_temperature'].astype('float16')
    df['precip_depth_1_hr'] = df['precip_depth_1_hr'].astype('float32')
    df['sea_level_pressure'] = df['sea_level_pressure'].astype('float32')
    df['wind_direction'] = df['wind_direction'].astype('float32')
    df['wind_speed'] = df['wind_speed'].astype('float16')
    df['square_feet'] = df['square_feet'].astype("float32")
    df['building_id'] = df['building_id'].astype("int16")